In [46]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.special import softmax
from scipy.stats import entropy
from scipy.stats import dirichlet
import pandas as pd

In [47]:
class GenerativeModel:
    """
    Generative model class, this class represents an agent, it will observe outcomes and generate actions.
    """
    def __init__(self, size=None, starting_A_alphas=None, starting_B_alphas=None,
                 expected_outcomes=None, starting_state=None):
        """
        :param size: Number of possible states the agent can be in
        :param starting_A_alphas: 2-dimensional list of alphas that inform dirichlet distributions from which
                                    priors over outcomes given states are generated
        :param starting_B_alphas: 3-dimensional list of alphas that inform dirichlet distributions from which
                                    priors over states given action and states are generated.
        :param expected_outcomes: List specifying the expected outcomes of this agent
        :param starting_state: Which state(s) the agent thinks it is in at the start of its lifespan
        """

        if size is None:
            size = 5

        if starting_A_alphas is None:
            starting_A_alphas = np.ones((size, 5))
        predicted_outcomes_given_state = np.zeros((size, size))

        # Generate probability distributions
        for s in range(size):
            predicted_outcomes_given_state[s] = dirichlet.mean(starting_A_alphas[s])

        if starting_B_alphas is None:
            starting_B_alphas = np.ones((3, size, size))
        predicted_future_states_given_previous_state_and_action = np.zeros((3, size, size))

        # Generate probability distributions
        for a in range(starting_B_alphas.shape[0]):
            for s in range(starting_B_alphas.shape[1]):
                predicted_future_states_given_previous_state_and_action[a][s] = dirichlet.mean(starting_B_alphas[a][s])

        if expected_outcomes is None:
            expected_outcomes = [0.2] * 5

        if starting_state is None:
            starting_state = [0.2, 0.2, 0.2, 0.2, 0.2]

        self.A_alphas = starting_A_alphas.copy()  # The A alphas inform the predicted outcomes given a state
        self.B_alphas = starting_B_alphas.copy()  # The B alphas inform the predicted state given an action and state
        self.starting_A_alphas = starting_A_alphas
        self.starting_B_alphas = starting_B_alphas

        self.predicted_outcomes_given_state = predicted_outcomes_given_state  # A: Predicted outcomes given the state
        self.predicted_future_states_given_previous_state_and_action = predicted_future_states_given_previous_state_and_action  # B: Predicted future state given the current state and the control states(?)
        self.expected_outcomes = np.array(expected_outcomes)  # C: Preferred outcomes given the model
        self.current_state = starting_state  # D: Starting state given the model
        self.Actions = [0, 1, 2]

        # Remember which action was taken last timestep, this is important for learning the state transition function
        self.action_taken = 0

    # Actions
    def act(self):
        """
        Generate an action
        :return: The action the agent chooses
        """
        action_qualities = []
        for action in self.Actions:
            # Determine predicted future state based on this action and inferred current state
            predicted_state = np.dot(self.current_state,
                                     self.predicted_future_states_given_previous_state_and_action[action])
            # Determine action quality
            predicted_quality = self.determine_quality(predicted_state, self.predicted_outcomes_given_state,
                                                       self.expected_outcomes)
            action_qualities.append(predicted_quality)

        action_probabilities = softmax(np.array(action_qualities))  # Change action qualities into probabilities
        best_action = np.random.choice(self.Actions, p=action_probabilities)
        self.action_taken = best_action  # Remember chosen action
        return best_action

    # Observations
    def observe(self, outcome):
        """
        Observe the outcome
        :param outcome: The outcome an agent observes
        :return: The state an agent infers it is in and the surprise (prediction error) the agent experienced at the
                    observation, these are for data storing purposes mainly
        """
        # Infer state of the agent
        predicted_state = self.aproximate_bayesian_inference(outcome)
        number_of_states = len(predicted_state)

        surprise = entropy(outcome, np.dot(predicted_state, self.predicted_outcomes_given_state))

        # Update distribution parameters based on outcome and inferred state, weighted by the surprise
        self.A_alphas += np.multiply(np.resize(outcome, (5, 5)).T, predicted_state).T * surprise
        self.B_alphas[self.action_taken] += (np.resize(self.current_state, (
        number_of_states, number_of_states)).T * predicted_state) * surprise

        # Update P(S|O) and P(S_t| A_{t-1}, S_{t-1})
        for s in range(number_of_states):
            self.predicted_future_states_given_previous_state_and_action[self.action_taken][s] = dirichlet.mean(
                self.B_alphas[self.action_taken][s])
            self.predicted_outcomes_given_state[s] = dirichlet.mean(self.A_alphas[s])

        # Update current state of the agent
        self.current_state = predicted_state
        return predicted_state, surprise

    def determine_quality(self, pred_state, pred_outcome_given_state, expected_outcome):
        """
        Determine the quality of a predicted state
        :param pred_state: Predicted state
        :param pred_outcome_given_state: Predicted outcomes given a state
        :param expected_outcome: Expected outcomes
        :return:
        """
        pred_state = np.array(pred_state)
        pred_outcome_given_state = np.array(pred_outcome_given_state)
        expected_outcome = np.array(expected_outcome)

        # Determine predicted outcome
        pred_outcome = np.dot(pred_state, pred_outcome_given_state)

        # Calculate predicted uncertainty as the expectation
        # of the entropy of the outcome, weighted by the
        # probability of that outcome
        pred_ent = np.sum(pred_state * entropy(pred_outcome_given_state, axis=1))

        # Calculate predicted divergence as the Kullback-Leibler
        # divergence between the predicted outcome and the expected outcome
        pred_div = entropy(pk=pred_outcome, qk=expected_outcome)

        # Return the sum of the negatives of these two
        return -pred_ent - pred_div

    def aproximate_bayesian_inference(self, observed_outcome):
        """
        Infer the state of the agent using an approximation of bayesian inference
        :param observed_outcome: The outcome observed by the agent
        :return: A probability distribution over states
        """
        observed_outcome = np.array(observed_outcome)
        expected_outcomes_given_state = np.array(self.predicted_outcomes_given_state)
        previous_action = self.action_taken
        previous_state = np.array(self.current_state)
        expected_future_states_given_control = np.array(self.predicted_future_states_given_previous_state_and_action)

        inferred_state = softmax(np.log(np.dot(expected_outcomes_given_state, observed_outcome)) + np.log(
            np.dot(previous_state, expected_future_states_given_control[previous_action])))
        return inferred_state

In [48]:
class GenerativeProcess:
    """
    Generative process class, responsible for generating observations and conforming to agent actions
    """
    def __init__(self, nr_of_possible_states=None, nr_of_possible_outcomes=None,
                 p_outcomes_given_state=None,
                 p_states_given_action_and_state=None, starting_states=None,
                 nr_of_agents=1):
        """
        Initialization function for the generative process
        :param nr_of_possible_states: Number of possible states an agent can be in
        :param nr_of_possible_outcomes: Number of possible outcomes an agent can observe
        :param p_outcomes_given_state: Probability of each outcome given a state
        :param p_states_given_action_and_state: Probability of each state given an action and state
        :param starting_states: True state in which each agent starts
        :param nr_of_agents: Number of agents
        """
        if nr_of_possible_states is None:
            nr_of_possible_states = 5

        if nr_of_possible_outcomes is None:
            nr_of_possible_outcomes = 5

        if p_states_given_action_and_state is None:
            p_states_given_action_and_state = self.create_transition_matrix(nr_of_possible_states)

        if p_outcomes_given_state is None:
            p_outcomes_given_state = np.zeros((nr_of_possible_states, nr_of_possible_outcomes))
            p_outcomes_given_state[0, 0] = 1  # Always get nothing in the DR
            p_outcomes_given_state[1:nr_of_possible_states] = [
                                                                          1 / nr_of_possible_outcomes] * nr_of_possible_outcomes

        if starting_states is None:
            starting_states = [[1, 0, 0, 0, 0]] * nr_of_agents

        self.agent_states = starting_states

        self.p_outcomes_given_state = p_outcomes_given_state  # A: Expected outcomes given the state
        self.p_future_states_given_control = p_states_given_action_and_state  # B: Expected future state given the current state and the control states(?)
        self.actions = [0, 1, 2]

    def act(self, agent_i):
        """
        Generate an observation and an outcome for an agent
        :param agent_i: Agent index, this is required because the generative process keeps track of the true state of
                        every agent separately
        :return: An outcome based on the agent's true state
        """
        agent_state = self.agent_states[agent_i]

        # Sample an outcome
        outcome = np.random.multinomial(1, np.dot(agent_state,
                                                  self.p_outcomes_given_state))
        return outcome

    # Observations
    def observe(self, action, agent_i):
        """
        Observe an action made by an agent
        :param action: The action made by an agent
        :param agent_i: Agent index, this is required because the generative process keeps track of the true state of
                        every agent separately
        """
        # Sample agent state
        future_state = np.random.multinomial(1, np.dot(self.agent_states[agent_i],
                                                       self.p_future_states_given_control[action]))
        self.agent_states[agent_i] = future_state  # Update agent state

    def create_transition_matrix(self, size=5):
        """
        Create Default state transition matrix
        :param size: Number of possible states agent can be in
        :return: Default state transition matrix
        """
        transition_matrix = np.zeros((2, size, size))
        transition_matrix[0, :, 0] = 0.9
        transition_matrix[0, :, 1:size] = 0.025
        transition_matrix[1, :, 0] = 0.05
        transition_matrix[1, :, 1:size] = 0.95 / (size - 1)
        return transition_matrix

In [49]:
class EvolutionaryFunction:
    """
    Evolutionary function class, responsible for creating new generations of agents and keeping track of agent data
    """

    def __init__(self, nr_of_agents=20, agent_nr_of_possible_states=5, utilities_per_state=None,
                 agent_expected_outcomes=None,
                 agents=None):
        """
        Initialization function of the EvolutionaryFunction
        :param nr_of_agents: Number of agents per generation
        :param agent_nr_of_possible_states: Number of possible states an agent can be in
        :param utilities_per_state: List specifying the evolutionary utility per state
        :param agent_expected_outcomes:  List specifying the default expected outcomes of agents
        :param agents: List of agents that takes the form of [[agents], [agent utilities]]
        """
        if utilities_per_state is None:
            utilities_per_state = [0, 1, 2, 3, 4]  # Default evolutionary utility per state

        if agent_expected_outcomes is None:
            agent_expected_outcomes = [0.2, 0.2, 0.2, 0.2, 0.2]  # By default an agent does not expect anything

        if agents is None:
            self.agents = []  # In this case agents should be created using the create_new_generation() function
            self.agent_utilities = []
        else:
            self.agents = agents[0]
            self.agent_utilities = agents[1]

        self.nr_of_agents = nr_of_agents
        self.agent_nr_of_possible_states = agent_nr_of_possible_states
        self.agent_actions = []
        self.utilities_per_state = utilities_per_state
        self.current_generation_states = [0, 0, 0, 0, 0]
        self.agent_expected_outcomes = agent_expected_outcomes

        # Generational Data collection
        # Comments show default values, same values with which simulations were run
        self.utility_per_generation = []  # [0: Total comulative, 1: average utlity, 2: most utility]
        self.actions_per_generation = []  # [0: Proportion left, 1: Proportion stay, 2: proportion right]
        self.expected_outcomes_per_generation = []  # [p(o_0), p(o_1), p(o_2), p(o_3), p(o_4)]
        self.surprise_per_generation = []  # [Average surprise per agent per generation]
        self.agents_per_generation = []  # [By default always same as nr_of_agents]
        self.states_per_generation = []  # [state occupation per agent per generation]

    def create_new_generation(self, new_generation=None):
        """
        For creating either a blank-slate generation or setting a new generation with a list of agents
        :param new_generation: optional list of agents
        """
        # By default this happens
        if new_generation == None:
            self.agents = [
                GenerativeModel(size=self.agent_nr_of_possible_states, expected_outcomes=self.agent_expected_outcomes)
                for i in range(self.nr_of_agents)]  # Create a new blank-slate generation
            self.agent_utilities = np.ones(self.nr_of_agents)
            self.agent_actions = np.zeros((self.nr_of_agents, 3))  # Assuming 3 possible actions

        # If we have already created a new generation (using select_probabilistically() for example)
        else:
            self.agents = new_generation
            self.agent_utilities = np.zeros(len(new_generation))
            self.agent_actions = np.zeros((self.nr_of_agents, 3))  # Assuming 3 possible action
            self.current_generation_states = [0, 0, 0, 0, 0]

    def select_probabilistically(self, N, killoff=None):
        """
        Probabilistically selects parents for a new generation of agents, then creates the agents.
        :param N: Number of agents
        :param killoff: A number specifying how many of the bottom agents to kill off (0% chance of reproduction)
        :return: A new generation of agents
        """
        agent_utilities = np.array(self.agent_utilities)  # For normal evolutionary pressure
        total_utility = np.sum(agent_utilities)
        p = agent_utilities / total_utility

        # if killoff is None:  # Uncomment this for heightened evolutionary pressure
        #     killoff = round(N / 10)  # Kill off the bottom (least evolutionary utility) 10% of agents
        # eligible_agents = np.argsort(agent_utilities)[killoff:]  # Select agents which may reproduce
        # total_utility = np.sum(agent_utilities[eligible_agents])
        # p = np.zeros(N)
        # p[eligible_agents] = agent_utilities[eligible_agents] / total_utility  # Specify the probabilities of
        # # reproducing for each agent


        new_generation = []
        parents = np.random.choice(a=np.arange(N), size=N, p=p)  # Select parents
        # parents = np.append(parents, np.argmax(agent_utilities))  # Optional: Best performing agent always reproduces

        for p in parents:
            # Mutate Prior over outcomes given state
            new_predicted_outcomes_given_state = self.mutate_predicted_outcomes_given_state(
                self.agents[p].starting_A_alphas) + \
                                                 self.agents[p].A_alphas / \
                                                 np.sum(self.agents[p].A_alphas, axis=1)[
                                                     ..., None] * 5 # Normalize and multiply by 5 so that ending alphas have limited influence on the starting alphas

            # Mutate Prior over state given action and state
            new_state_transition_function = self.mutate_state_transition_function(self.agents[p].starting_B_alphas) + \
                                            self.agents[p].B_alphas / \
                                            np.sum(self.agents[p].B_alphas, axis=2)[
                                                ..., None] * 5  # Normalize and multiply by 5 so that ending alphas have limited influence on the starting alphas

            # Mutate Expected outcomes
            new_expected_outcomes = self.mutate_expected_outcomes(self.agents[p].expected_outcomes)

            # Add child to the new generation
            new_generation += [
                GenerativeModel(expected_outcomes=new_expected_outcomes,
                                starting_B_alphas=new_state_transition_function,
                                starting_A_alphas=new_predicted_outcomes_given_state)]
        return new_generation

    def mutate_predicted_outcomes_given_state(self, predicted_outcomes_given_state, mu=0, sigma=0.5):
        """
        Mutates the given list of predicted outcomes per state by adding a sample from a normal distribution
        to the probability of each state, then normalizing.
        :param predicted_outcomes_given_state: Must be a 2-dimensional list or ndarray
        :param sigma: standard deviation
        :param mu: mean
        :return:
        """
        predicted_outcomes_given_state = np.array(predicted_outcomes_given_state)
        new_predicted_outcomes_given_state = predicted_outcomes_given_state + np.random.normal(mu, sigma,
                                                                                               predicted_outcomes_given_state.shape)
        new_predicted_outcomes_given_state[
            new_predicted_outcomes_given_state <= 0] = 0.001  # Alphas must be larger than 0
        new_predicted_outcomes_given_state = new_predicted_outcomes_given_state / \
                                             np.sum(new_predicted_outcomes_given_state, axis=1)[
                                                 ..., None] * 5  # Multiple by 5 and normalize so that alpha values do not grow too large
        return new_predicted_outcomes_given_state

    def mutate_state_transition_function(self, state_transition_function, mu=0, sigma=0.5):
        """
        Mutates the given list of predicted states per action and state by adding a sample from a normal distribution
        to the probability of each state, then normalizing.
        :param state_transition_function: Must be a 3-dimensional list or ndarray
        :param sigma: standard deviation
        :param mu: mean
        :return: Mutated version of the state transition function
        """
        state_transition_function = np.array(state_transition_function)
        new_state_transition_function = state_transition_function + np.random.normal(mu, sigma,
                                                                                     state_transition_function.shape)
        new_state_transition_function[new_state_transition_function <= 0] = 0.001  # Alphas must be larger than 0
        new_state_transition_function = new_state_transition_function / \
                                        np.sum(new_state_transition_function, axis=2)[
                                            ..., None] * 5  # Multiple by 5 and normalize so that alpha values do not grow too large

        return new_state_transition_function

    def mutate_expected_outcomes(self, expected_outcomes, mu=0, sigma=0.02):
        """
        Mutates the given list of expected outcomes by adding a sample from a normal distribution to the probability
        of each outcome, then normalizing.
        :param expected_outcomes: Must be a 1-dimensional list or ndarray
        :param sigma: standard deviation
        :param mu: mean
        :return: Mutated version of expected outcomes
        """
        expected_outcomes = np.array(expected_outcomes)
        new_expected_outcomes = np.clip(expected_outcomes + np.random.normal(mu, sigma, expected_outcomes.shape), 0.001,
                                        1)
        norm = np.sum(new_expected_outcomes)  # Normalization
        new_expected_outcomes /= norm
        return new_expected_outcomes

    def evolutionary_utility(self, outcome):
        """
        Returns the utility of a certain outcome
        :param outcome: array specifying outcome(s)
        :return: evolutionary utility of outcomes
        """
        utility = np.dot(self.utilities_per_state, outcome)
        return utility

    def store_generation_data(self):
        """
        Store relevant data of this generation for plotting purposes
        """
        total_utility = np.sum(self.agent_utilities)
        average_utility = total_utility / len(self.agent_utilities)
        most_utility = np.max(self.agent_utilities)
        self.utility_per_generation.append([total_utility, average_utility, most_utility])
        self.actions_per_generation.append(np.sum(self.agent_actions, axis=0) / np.sum(
            self.agent_actions))  # Only store the average proportion of actions
        self.expected_outcomes_per_generation.append(
            self.get_expected_outcomes())  # Only store the average expected outcomes
        self.agents_per_generation.append(
            self.nr_of_agents)  # Normally this should always be the same, may not be for different ways of generation creation

    def get_expected_outcomes(self):
        """
        Get the average expected outcomes of the current generation
        :return: Average expected outcomes per agent
        """
        expected_outcomes = np.zeros((self.nr_of_agents, self.agents[0].expected_outcomes.shape[0]))
        for i, a in enumerate(self.agents):
            expected_outcomes[i] = a.expected_outcomes
        return expected_outcomes

    def get_avg_expected_outcomes(self):
        """
        Get the average expected outcomes of the current generation
        :return: Average expected outcomes per agent
        """
        average_expected_outcomes = np.zeros(self.agent_nr_of_possible_states)
        for a in self.agents:
            average_expected_outcomes += a.expected_outcomes
        average_expected_outcomes /= len(self.agents)
        return average_expected_outcomes


In [50]:
def Evolution_sandbox(T=20, G=2, A=1, size=5, GP=None, EF=None, cont=False):
    """
    This function runs the evolution simulation
    :param T: Number of timesteps per generation
    :param G: Number of generations
    :param A: Number of agents per generation
    :param size: Number of possible states the agent can be in
    :param GP: Generative Process
    :param EF: Evolutionary Function
    :param cont: If this is true, the simulation will continue with the agents the Evolutionary Function already has,
                    otherwise a new simulation is run with a blank-slate first generation
    :return: The Evolutionary Function, the Evolutionary Function contains relevant data
    """
    if GP is None:
        GP = GenerativeProcess(nr_of_possible_states=size, nr_of_agents=A)
    if EF is None:
        EF = EvolutionaryFunction(nr_of_agents=A)

    for g in range(G):
        # print(g)
        agent_states_this_generation = []
        true_agent_states_this_generation = []

        # The first generation, determine if we start from scratch or continue with the agents the
        # EvolutionaryFunction already has.
        if g == 0 and not cont:
            EF.create_new_generation()

        EF.create_new_generation(EF.select_probabilistically(N=A))
        starting_states = generate_starting_locations(EF.nr_of_agents, size)
        GP.agent_states = starting_states
        true_agent_states_this_generation.append(np.sum(starting_states, axis=0) / EF.nr_of_agents)
        surprise_this_generation = []

        for t in range(T):
            agent_states_this_timestep = [0, 0, 0, 0, 0]
            true_agent_states_this_timestep = [0, 0, 0, 0, 0]
            surprise_this_timestep = []
            for a in range(EF.nr_of_agents):
                # Agent acts, environment 'observes'
                agent_action = EF.agents[a].act()
                EF.agent_actions[a][agent_action] += 1
                GP.observe(agent_action, a)

                # Environments 'acts', agent observes
                outcome = GP.act(a)
                inferred_state, surprise = EF.agents[a].observe(outcome)
                EF.agent_utilities[a] += [EF.evolutionary_utility(outcome)]

                # Collect timestep data
                agent_states_this_timestep += inferred_state
                true_agent_states_this_timestep += GP.agent_states[a]
                surprise_this_timestep += [surprise]
            surprise_this_generation += [surprise_this_timestep]

            # Collect data generational data
            agent_states_this_generation.append(np.array(agent_states_this_timestep).T / EF.nr_of_agents)
            true_agent_states_this_generation.append(np.array(true_agent_states_this_timestep).T / EF.nr_of_agents)

        EF.surprise_per_generation += [surprise_this_generation]
        EF.states_per_generation.append(true_agent_states_this_generation)
        EF.store_generation_data()

        # For plotting (Optional)

        # Plot surprise this generation
        plt.figure()
        plt.plot(np.mean(surprise_this_generation, axis=1))
        plt.title('Average surprise of generation over time ' + str(g))
        plt.ylim(-0.1, 4)
        plt.xlabel('Timestep')
        plt.ylabel('Surprise')
        # plt.savefig('Surprise' + str(g))
        plt.show()

        # Plot inferred agent states of this generation
        # take_X_timestep_average(10, agent_states_this_generation, 'Average inferred agent states of generation ' + str(g), 'State probability', 'Timestep')
        plt.figure()
        plt.plot(agent_states_this_generation)
        plt.title('Average inferred state of generation ' + str(g))
        plt.legend(['0', '1', '2', '3', '4'])
        plt.ylim(-0.1, 1.2)
        plt.xlabel('Timestep')
        plt.ylabel('state probability')
        # plt.savefig('Test inferred states' + str(g))
        plt.show()

        # Plot true agent states of this generation
        #         take_X_timestep_average(10, true_agent_states_this_generation, 'Average true state of generation ' + str(g) , 'state', 'Timestep')
        plt.figure()
        plt.plot(true_agent_states_this_generation)
        plt.title('Average true state of generation ' + str(g))
        plt.legend(['0', '1', '2', '3', '4'])
        plt.ylim(-0.1, 1.2)
        plt.xlabel('Timestep')
        plt.ylabel('state')
        # plt.savefig('Test true states' + str(g))
        plt.show()
    return EF


def generate_starting_locations(nr_of_agents, nr_of_possible_states, p=None):
    """
    Generate semi-random starting locations for a number of agents
    :param nr_of_agents: Number of agents
    :param nr_of_possible_states: Number of possible states an agent can be in
    :param p: Probability of each state
    :return: A list of starting locations
    """
    if p is None:
        p = np.ones(nr_of_possible_states) / nr_of_possible_states
    starting_states = []
    for a in range(nr_of_agents):
        starting_states.append(np.random.multinomial(1, p))
    return starting_states


def single_agent(T=1000, GP=None, agent=None, utility=None):
    """
    Function for examining the behaviour of a singular agent
    :param T: Number of timesteps the agent will 'live'
    :param GP: Generative Process
    :param agent: The agent
    :param utility: Utility per outcome
    :return: The agent
    """
    if GP is None:
        GP = GenerativeProcess(nr_of_agents=1)
    if agent is None:
        agent = GenerativeModel()
    if utility is None:
        utility = [0, 2, 4, 6, 8]

    starting_states = generate_starting_locations(1, 5)
    GP.agent_states = [starting_states[0]]
    agent_state = []
    agent_actions = []
    agent_utility = []
    true_agent_state = [starting_states[0]]
    observed_outcomes = []
    agent_surprise = []
    for t in range(T):
        # Agent acts, environment 'observes'
        agent_action = agent.act()
        GP.observe(agent_action, 0)

        # Environments 'acts', agent observes
        outcome = GP.act(0)
        inferred_state, surprise = agent.observe(outcome)

        agent_utility += [[np.dot(utility, outcome)]]

        x = np.zeros(3)
        x[agent_action] = 1
        agent_actions.append(x)
        agent_state.append(inferred_state)
        observed_outcomes.append(outcome)
        agent_surprise.append([surprise])
        true_agent_state.append(GP.agent_states[0])

    avg_t = 1000
    take_X_timestep_average(avg_t, agent_surprise, str(avg_t) + ' Timestep average surprise', 'Timestep', 'Surprise',
                            ylim=(-0.1, 2), legend=['agent utility'])
    take_X_timestep_average(avg_t, agent_utility, str(avg_t) + ' Timestep average utility', 'Timestep', 'Utility',
                            ylim=(-0.1, 20), legend=['agent utility'])
    take_X_timestep_average(avg_t, agent_actions, str(avg_t) + ' Timestep average agent action', 'Timestep',
                            'Action proportion', legend=['left', 'stay', 'right'])
    take_X_timestep_average(avg_t, agent_state, str(avg_t) + ' Timestep average inferred agent state', 'Timestep',
                            'State proportion')
    take_X_timestep_average(avg_t, true_agent_state, str(avg_t) + ' Timestep average true agent state', 'Timestep',
                            'State proportion')
    take_X_timestep_average(avg_t, observed_outcomes, str(avg_t) + ' Timestep average observed outcome', 'Timestep',
                            'Observation proportion',
                            legend=['outcome 0', 'outcome 1', 'outcome 2', 'outcome 3', 'outcome 4'])
    return agent


def take_X_timestep_average(X, values, title=None, xlabel=None, ylabel=None, ylim=None, legend=None):
    """
    Plot a sliding-window timestep average of some list of values
    :param X: Size of the window
    :param values: Some list of values
    :param title: Title of the plot
    :param xlabel: xlabel of the plot
    :param ylabel: ylabel of the plot
    :param ylim: ylim of the plot
    :param legend: legend of the plot
    :return: Returns the list of averages
    """
    if ylabel is None:
        ylabel = 'Y_value'
    if xlabel is None:
        xlabel = 'Timestep'
    if title is None:
        title = 'plot'
    if ylim is None:
        ylim = (-0.1, 1.2)
    if legend is None:
        legend = ['state 0', 'state 1', 'state 2', 'state 3', 'state 4']
    values = np.array(values)
    values = np.insert(values, 0, np.zeros([X, values.shape[1]]), axis=0)
    averages = []
    for t in range(values.shape[0]):
        averages.append(np.sum(values[t - X:t], axis=0) / X)

    plt.figure()
    plt.plot(averages[X:])
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.ylim(ylim[0], ylim[1])
    plt.legend(legend)
    plt.savefig('THESIS_not_modified ' + str(title))
    plt.show()
    return averages

In [51]:
def run_simulation():
    """
    Runs an evolution simulation,
    Change parameters in this function to adjust the simulation
    :return: Evolutionary Function
    """
    utilities = [1, 5, 25, 125, 625]  # Evolutionary utility of each outcome

    # P_gp(O|S)
    predicted_outcomes_given_state = np.array([[1, 0.0, 0.0, 0.0, 0.0],
                                               [0.05, 0.8, 0.05, 0.05, 0.05],
                                               [0.05, 0.05, 0.8, 0.05, 0.05],
                                               [0.05, 0.05, 0.05, 0.8, 0.05],
                                               [0.05, 0.05, 0.05, 0.05, 0.8]])

    # P_gp(S_{t+1}|A_t,S_t)
    predicted_future_states_given_control = np.array([[[0.1, 0.1, 0, 0, 0.8],
                                                       [0.8, 0.1, 0.1, 0, 0],
                                                       [0, 0.8, 0.1, 0.1, 0],
                                                       [0, 0, 0.8, 0.1, 0.1],
                                                       [0.1, 0, 0, 0.8, 0.1]],

                                                      [[0.9, 0.05, 0, 0, 0.05],
                                                       [0.05, 0.9, 0.05, 0, 0],
                                                       [0, 0.05, 0.9, 0.05, 0],
                                                       [0, 0, 0.05, 0.9, 0.05],
                                                       [0.05, 0, 0, 0.05, 0.9]],

                                                      [[0.1, 0.8, 0, 0, 0.1],
                                                       [0.1, 0.1, 0.8, 0, 0],
                                                       [0, 0.1, 0.1, 0.8, 0],
                                                       [0, 0, 0.1, 0.1, 0.8],
                                                       [0.8, 0, 0, 0.1, 0.1]]])

    T = 1825  # Number of timesteps each agent lives
    G = 100  # Number of generations
    A = 100  # Number of agetns per generation

    GP = GenerativeProcess(nr_of_agents=A, p_states_given_action_and_state=predicted_future_states_given_control,
                           p_outcomes_given_state=predicted_outcomes_given_state)
    EF = EvolutionaryFunction(nr_of_agents=A, utilities_per_state=utilities)
    EF = Evolution_sandbox(T=T, G=G, A=A, GP=GP, EF=EF)
    return EF

In [52]:
# EF = run_simulation()

In [53]:
def run_single_agent():
    """
    Runs a single agent simulation,
    Change parameters in this function to adjust the simulation
    :return: The agent after simulation
    """
    # P_gp(O|S)
    predicted_outcomes_given_state = np.array([[1, 0.0, 0.0, 0.0, 0.0],
                                               [0.05, 0.8, 0.05, 0.05, 0.05],
                                               [0.05, 0.05, 0.8, 0.05, 0.05],
                                               [0.05, 0.05, 0.05, 0.8, 0.05],
                                               [0.05, 0.05, 0.05, 0.05, 0.8]])

    # P_gp(S_{t+1}|A_t,S_t)
    predicted_future_states_given_control = np.array([[[0.1, 0.1, 0, 0, 0.8],
                                                       [0.8, 0.1, 0.1, 0, 0],
                                                       [0, 0.8, 0.1, 0.1, 0],
                                                       [0, 0, 0.8, 0.1, 0.1],
                                                       [0.1, 0, 0, 0.8, 0.1]],

                                                      [[0.9, 0.05, 0, 0, 0.05],
                                                       [0.05, 0.9, 0.05, 0, 0],
                                                       [0, 0.05, 0.9, 0.05, 0],
                                                       [0, 0, 0.05, 0.9, 0.05],
                                                       [0.05, 0, 0, 0.05, 0.9]],

                                                      [[0.1, 0.8, 0, 0, 0.1],
                                                       [0.1, 0.1, 0.8, 0, 0],
                                                       [0, 0.1, 0.1, 0.8, 0],
                                                       [0, 0, 0.1, 0.1, 0.8],
                                                       [0.8, 0, 0, 0.1, 0.1]]])
    A = 1

    GP = GenerativeProcess(nr_of_agents=A, p_states_given_action_and_state=predicted_future_states_given_control,
                           p_outcomes_given_state=predicted_outcomes_given_state)

    agent_A_alphas = (predicted_outcomes_given_state + 0.001)  # Starting A alphas of the agent
    agent_B_alphas = (predicted_future_states_given_control + 0.001)  # Starting B alphas of the agent

    expected_outcomes = [0.2, 0.2, 0.2, 0.2, 0.2]  # Expected outcomes of the agent
    agent = GenerativeModel(starting_A_alphas=agent_A_alphas, starting_B_alphas=agent_B_alphas,
                            expected_outcomes=expected_outcomes)

    agent = single_agent(T=10000, GP=GP, agent=agent)
    return agent



In [54]:
# agent = run_single_agent()

In [41]:
# Adapted from https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
def mean_confidence_interval(data, confidence=0.95):
    """
    Creates confidence intervals for the expected outcomes per generation.
    :data: 3-dimensional data with shape [G, A, O]
            G is the number of generations
            A is the number of agents per generations
            O is the number of possible outcomes
    :return: the means of the data,
                the means plus the positive edge of the confidence interval,
                and the means plus the negative edge of the confidence interval
    """
    a = 1.0 * np.array(data)
    n = a.shape[1]
    m, se = np.mean(a, axis=1), scipy.stats.sem(a, axis=1)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
    return m, m - h, m + h

In [42]:
def plot_data(EF, T=1000):
    """
    Plots the average expected outcomes per generation,
            the average surprise per generation,
            the average proportion of actions per generation,
            and the average utility per timestep per generation
    :param EF: Evolutionary Function
    :param T: Timesteps per generation
    """
    y = np.array(EF.expected_outcomes_per_generation)
    n = y.shape[0]
    x = range(n)

    plt.plot(x, np.mean(y, axis=1))
    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 0', 'Outcome 1', 'Outcome 2', 'Outcome 3', 'Outcome 4'])
    #     plt.savefig('Expected_outcomes_per_generation')
    plt.show()

    plt.plot(np.mean(np.mean(EF.surprise_per_generation, axis=1), axis=1))
    plt.title("Surprise per generation")
    plt.xlabel('Generation')
    plt.ylabel('Surprise')
    #     plt.savefig('Surprise_per_generation')
    plt.show()

    plt.plot(np.array(EF.actions_per_generation)[:, 0])
    plt.plot(np.array(EF.actions_per_generation)[:, 1])
    plt.plot(np.array(EF.actions_per_generation)[:, 2])
    plt.legend(['Left', 'Stay', 'Right'])
    plt.title('Proportion of actions per generation')
    plt.xlabel('Generation')
    plt.ylabel('Action proportion')
    #     plt.savefig('Agent_actions')
    plt.show()

    plt.plot(np.array(EF.utility_per_generation)[:, 1] / T)
    plt.title('Average utility per timestep per generation')
    plt.xlabel('Generation')
    plt.ylabel('Average utility')
    #     plt.savefig('Average_utility')
    plt.show()

In [43]:
def plot_expected_outcomes(EF):
    """
    Plots the average expected outcomes per generation with (by default) 95% confidence intervals
    :param EF: Evolutionary function
    """
    y = np.array(EF.expected_outcomes_per_generation)
    n = y.shape[0]
    x = range(n)

    plt.plot(x, np.mean(y, axis=1))
    plt.plot(x, np.ones(n) * 0.2, color='black')
    # Outcome 0
    mean, lower, upper = mean_confidence_interval(y[:, :, 0])
    ci_m = np.mean(y[:, :, 0], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, alpha=.1)

    mean, lower, upper = mean_confidence_interval(y[:, :, 1])
    ci_m = np.mean(y[:, :, 1], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, alpha=.1)

    mean, lower, upper = mean_confidence_interval(y[:, :, 2])
    ci_m = np.mean(y[:, :, 2], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, alpha=.1)

    mean, lower, upper = mean_confidence_interval(y[:, :, 3])
    ci_m = np.mean(y[:, :, 3], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, alpha=.1)

    mean, lower, upper = mean_confidence_interval(y[:, :, 4])
    ci_m = np.mean(y[:, :, 4], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, alpha=.1)

    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 0', 'Outcome 1', 'Outcome 2', 'Outcome 3', 'Outcome 4', 'baseline'])
    #     plt.savefig('Confidence_intervals')
    plt.show()

In [44]:
def plot_expected_outcomes_seperate(EF):
    """
    Plots the average expected outcomes per generation with (by default) 95% confidence intervals,
    each outcome is plotted separately.
    :param EF: Evolutionary Function
    """
    y = np.array(EF.expected_outcomes_per_generation)
    n = y.shape[0]
    x = range(n)

    # Outcome 0
    plt.plot(x, np.mean(y[:, :, 0], axis=1))
    plt.plot(x, np.ones(n) * 0.2, color='black')
    mean, lower, upper = mean_confidence_interval(y[:, :, 0])
    ci_m = np.mean(y[:, :, 0], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, alpha=.1)
    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 0'])
    plt.ylim([0, 1])
    # plt.savefig('Confidence_interval_outcome_0')
    plt.show()

    # Outcome 1
    plt.plot(x, np.mean(y[:, :, 1], axis=1), color='orange')
    plt.plot(x, np.ones(n) * 0.2, color='black')
    mean, lower, upper = mean_confidence_interval(y[:, :, 1])
    ci_m = np.mean(y[:, :, 1], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, color='orange', alpha=.1)
    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 1'])
    plt.ylim([0, 1])
    # plt.savefig('Confidence_interval_outcome_1')
    plt.show()

    # Outcome 2
    plt.plot(x, np.mean(y[:, :, 2], axis=1), color='green')
    plt.plot(x, np.ones(n) * 0.2, color='black')
    mean, lower, upper = mean_confidence_interval(y[:, :, 2])
    ci_m = np.mean(y[:, :, 2], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, color='green', alpha=.1)
    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 2'])
    plt.ylim([0, 1])
    # plt.savefig('Confidence_interval_outcome_2')
    plt.show()

    # Outcome 3
    plt.plot(x, np.mean(y[:, :, 3], axis=1), color='red')
    plt.plot(x, np.ones(n) * 0.2, color='black')
    mean, lower, upper = mean_confidence_interval(y[:, :, 3])
    ci_m = np.mean(y[:, :, 3], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, color='red', alpha=.1)
    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 3'])
    plt.ylim([0, 1])
    # plt.savefig('Confidence_interval_outcome_3')
    plt.show()

    # Outcome 4
    plt.plot(x, np.mean(y[:, :, 4], axis=1), color='purple')
    plt.plot(x, np.ones(n) * 0.2, color='black')
    mean, lower, upper = mean_confidence_interval(y[:, :, 4])
    ci_m = np.mean(y[:, :, 4], axis=1)
    plt.fill_between(x, ci_m - lower, ci_m + upper, color='purple', alpha=.1)
    plt.title("Expected outcomes per generation")
    plt.xlabel('Generation')
    plt.ylabel('Outcome probability')
    plt.legend(['Outcome 4'])
    plt.ylim([0, 1])
    # plt.savefig('Confidence_interval_outcome_4')
    plt.show()